In [ ]:
# !pip install langchain
# !pip install openai
#!pip install PyPDF2
# !pip install faiss-cpu
# !pip install tiktoken
#!pip install --upgrade pip
!pip install -U langchain-community
#!pip install python-dotenv

In [77]:

!pip freeze > requirements.txt


In [42]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [78]:
from dotenv import load_dotenv
import os

# Carica le variabili di ambiente dal file .env
load_dotenv()

# Ora puoi accedere alle variabili di ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [79]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('hoder.pdf')

In [80]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [81]:
raw_text

'Multilingual constructions\nA diasystematic approach to common structures*\nSteffen Höder\nUniversity of Münster, Germany\nLanguage contact phenomena are often described with reference to their effect \non the monolingual systems of the varieties involved, both in historical and \nin contact linguistics. This contribution argues that an essentially multilingual \nperspective on these phenomena is more adequate. Bilingual speakers in stable bilingual groups create a common system for all their languages, incorporating \nboth interlingual links and language-unspecified elements along with language-\nspecific structures. In a construction grammar analysis, such systems as well as changes within this type of system can be conceptualized as interlingual \nconstructional networks, which are established, stored, and processed in exactly \nthe same way as monolingual grammars.\nKeywords : German, Danish, Old Swedish, Latin, multilingualism, diasystem, \nconstruction grammar\n1. Introduction\n

In [82]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

Created a chunk of size 917, which is longer than the specified 800
Created a chunk of size 825, which is longer than the specified 800
Created a chunk of size 869, which is longer than the specified 800
Created a chunk of size 941, which is longer than the specified 800


In [83]:
len(texts)

81

In [84]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [85]:
document_search = FAISS.from_texts(texts, embeddings)

In [86]:
document_search


In [87]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [88]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [116]:
query = "Quali sono gli obiettivi della ricerca? scrivi almeno tre capoversi"
docs = document_search.similarity_search(query)
output = chain.run(input_documents=docs, question=query)  # Salva l'output in una variabile

# Scrivi la domanda e la risposta nel file, separati da una riga vuota, e metti la domanda tra ##
with open('output.txt', 'w', encoding='utf-8') as file:
    file.write('\n\n' + '## ' + query + ' ##' + '\n\n' + output)
# Stampa l'output a video
print('\n\n' + '## ' + query + ' ##' + '\n\n' + output)



## Quali sono gli obiettivi della ricerca? scrivi almeno tre capoversi ##


1. Il primo obiettivo della ricerca è quello di indagare sul problema teorico centrale della linguistica di contatto: se sia ragionevole assumere che le lingue o le varietà usate da un gruppo di parlanti multilingue rimangano sistemi distinti anche in casi di contatto linguistico intenso e stabile, oppure se sia più appropriato considerarle come un unico sistema comune.
2. Un secondo obiettivo è quello di esplorare come il contatto linguistico influenzi i cambiamenti nelle lingue coinvolte e come questi cambiamenti siano storicamente stabili o meno.
3. Infine, la ricerca mira a comprendere il fenomeno del contatto linguistico da una prospettiva multilingue, osservando come l'interazione cognitiva tra le lingue coinvolte influenzi i fenomeni di trasferimento e cambiamento linguistico.


In [117]:
# Seconda domanda
query2 = "Quali sono le metodologie della ricerca? alemeno tre capoversi"
docs2 = document_search.similarity_search(query2)
output2 = chain.run(input_documents=docs2, question=query2)

# Aggiungi la seconda domanda e la risposta al file, separati da una riga vuota, e metti la domanda tra ##
with open('output.txt', 'a', encoding='utf-8') as file:
    file.write('\n\n' + '##' + query2 + '##' + '\n\n' + output2 + '\n\n')
    
# Stampa l'output a video
print('\n\n' + '## ' + query2 + '##' + '\n\n' + output2 + '\n\n')



## Quali sono le metodologie della ricerca? alemeno tre capoversi##


1. La ricerca sui multilinguismi si basa su una varietà di metodologie, tra cui l'analisi linguistica, lo studio delle differenze linguistiche e culturali, e l'osservazione dei comportamenti comunicativi dei multilingui.
2. Alcune delle principali metodologie utilizzate sono l'analisi della struttura grammaticale, l'analisi dei sistemi di suoni e la comparazione delle parole in diverse lingue.
3. Altre metodologie includono lo studio dei modi in cui i multilingui gestiscono le loro lingue e il loro uso del codice misto, nonché l'analisi dei cambiamenti nella lingua causati dal contatto con altre lingue.




In [118]:
#terza domanda
query3 = "Quali sono i risultati della ricerca? alemeno tre capoversi"
docs3 = document_search.similarity_search(query3)
output3 = chain.run(input_documents=docs3, question=query3)

# Aggiungi la seconda domanda e la risposta al file, separati da una riga vuota, e metti la domanda tra ##
with open('output.txt', 'a', encoding='utf-8') as file:
    file.write('\n\n' + '##' + query3 + '##' + '\n\n' + output3 + '\n\n')

# Stampa l'output a video
print('\n\n' + '##' + query3 + '##' + '\n\n' + output3 + '\n\n')



##Quali sono i risultati della ricerca? alemeno tre capoversi##


Non è possibile rispondere con precisione a questa domanda senza ulteriori informazioni. La ricerca menzionata sembra trattare di diversi aspetti della lingua svedese antica e della sua relazione con il latino, ma non è chiaro quali risultati specifici siano stati ottenuti. Inoltre, la domanda potrebbe riferirsi a diverse parti del testo sopra riportato, senza specificare quale. Senza ulteriori informazioni, non è possibile fornire una risposta precisa. 




In [122]:
from docx import Document
from docx.shared import Pt

def convert_txt_to_docx(input_txt_file, output_docx_file):
    # Creazione di un nuovo documento Word
    document = Document()
    
    # Lettura del file di testo
    with open(input_txt_file, 'r', encoding='utf-8') as file:
        for line in file:
            # Rimuovi spazi bianchi iniziali e finali
            stripped_line = line.strip()
            
            # Identificazione dei titoli
            if stripped_line.startswith('##'):
                # Rimuovi i segni "##" e aggiungi il titolo al documento
                title = stripped_line[2:].strip()
                heading = document.add_heading(level=1)
                run = heading.add_run(title)
                run.font.size = Pt(14)  # Imposta la dimensione del carattere del titolo
            else:
                # Aggiungi il paragrafo al documento
                document.add_paragraph(stripped_line)
    
    # Salvataggio del documento
    document.save(output_docx_file)

# Esempio di utilizzo
input_txt_file = 'output.txt'
output_docx_file = 'output.docx'
convert_txt_to_docx(input_txt_file, output_docx_file)